<center><h1>Conway's Game of Life in Python</h1></center>

In 1970 the British Mathematician John Conway created his "Game of Life" -- a set of rules that mimics the chaotic yet patterned growth of a colony of biological organisms. The "game" takes place on a two-dimensional grid consisting of "living" and "dead" cells, and the **rules to step from generation to generation are simple:**

* **Overpopulation**: if a living cell is surrounded by more than three living cells, it dies.
* **Stasis**: if a living cell is surrounded by two or three living cells, it survives.
* **Underpopulation**: if a living cell is surrounded by fewer than two living cells, it dies.
* **Reproduction**: if a dead cell is surrounded by exactly three cells, it becomes a live cell

By enforcing these rules in sequential steps, beautiful and unexpected patterns can appear.

Here I'll use Python and NumPy to compute generational steps for the game of life.

Because the Game of Life is so simple, the time step can be computed rather tersely in Python.  Two implementation possibilities: one using **generator expressions**, and one using the **convolve2d** function from scipy.

**<span style="color:Red">If you find this kernel useful, Please Upvote it , it motivates me to write more Quality content**

## Helpful functions

* credits http://jakevdp.github.io/pages/about.html

In [ ]:
!pip install -q JSAnimation

In [ ]:
%matplotlib inline

In [ ]:
# JSAnimation import available at https://github.com/jakevdp/JSAnimation
from JSAnimation.IPython_display import display_animation, anim_to_html
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.image as image
from IPython.display import HTML

* Helper matplotlib animation function

In [ ]:
def animate_fig(X,cmap='binary',frames=10, interval=300): 
    
    X_blank = np.zeros_like(X)
    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(figsize=(38,38), dpi=10)
    ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
    im = ax.imshow(X,cmap=cmap)
    im.set_clim(-0.05, 1) 

    def init():
        im.set_data(X_blank)
        return (im,)

    # animation function.  This is called sequentially
    def animate(i):
        im.set_data(animate.X)
        animate.X = life_step(animate.X)
        return (im,)

    animate.X = X

    ### call the animator. blit=True means only re-draw the parts that have changed.

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=interval, blit=True)

    # equivalent to rcParams['animation.html'] = 'html5'
    rc('animation', html='jshtml')
    rc
    return HTML(anim.to_jshtml())

In [ ]:
shape = (25,25)

### Helper functions for life step according to above 4 game rules

In [ ]:
def life_step_1(X):
    """Game of life step using generator expressions"""
    nbrs_count = sum(np.roll(np.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0))
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

def life_step_2(X):
    """Game of life step using scipy tools"""
    from scipy.signal import convolve2d
    nbrs_count = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

In [ ]:
life_step = life_step_1

In [ ]:
df = pd.read_csv('../input/conways-reverse-game-of-life-2020/train.csv')
test = pd.read_csv('../input/conways-reverse-game-of-life-2020/test.csv')

In [ ]:
df.head()

In [ ]:
def plot_fig(n): # n = row of train data
    X = df.iloc[n][2:625+2].values
    Y = df.iloc[n][625+2:].values
    shape = (25,25)
    plt.figure(1,figsize=(16,6))
    plt.subplot(121)
    plt.imshow(X.reshape(shape),cmap='binary');plt.title('start stage')
    plt.subplot(122)
    plt.imshow(Y.reshape(shape),cmap='binary');plt.title('stop stage')
    plt.show()

# Helper functions to Animate

In [ ]:
def plot_animate_fig(idx,cmap='binary'):  # Enter row number
    
    X = df.iloc[idx][2:625+2].values
    Y = df.iloc[idx][625+2:].values
    shape = (25,25)
    plt.figure(1,figsize=(14,6))
    plt.subplot(121)
    plt.imshow(X.reshape(shape),cmap=cmap);plt.title('start stage')
    plt.subplot(122)
    plt.imshow(Y.reshape(shape),cmap=cmap);plt.title('stop stage')
    plt.show()
    
    print('Approximated Animation till stop stage : ')
    
    X = df.iloc[idx][2:625+2].values
    X = X.reshape(shape)
    fr = df.iloc[idx][1]
    X_blank = np.zeros_like(X)
    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(figsize=(38,38), dpi=10)
    ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
    im = ax.imshow(X,cmap=cmap)
    im.set_clim(-0.05, 1) 

    def init():
        im.set_data(X_blank)
        return (im,)

    # animation function.  This is called sequentially
    def animate(i):
        im.set_data(animate.X)
        animate.X = life_step(animate.X)
        return (im,)

    animate.X = X

    ### call the animator. blit=True means only re-draw the parts that have changed.

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=fr, interval=300, blit=True)

    # equivalent to rcParams['animation.html'] = 'html5'
    rc('animation', html='jshtml')
    rc
    return HTML(anim.to_jshtml())

# Lets grab a single row and see start and stop, and animate it.

In [ ]:
plot_animate_fig(53)

### Work in progress stay tuned

# Some simple oscillators (The "Blinker" and the "Toad")

An oscillator is a pattern that returns to its initial configuration after some number of steps. The static patterns shown above could be thought of as oscillators with a period of one. Here are two commonly-seen period-two oscillators:

In [ ]:
blinker  =  [ 1 ,  1 ,  1 ] 
toad  =  [[ 1 ,  1 ,  1 ,  0 ], 
        [ 0 ,  1 ,  1 ,  1 ]]

X = np.zeros((6, 11))
X[2, 1:4] = blinker
X[2:4, 6:10] = toad
animate_fig(X.astype(int),frames=4)

# Another Oscillator: The "Pulsar"

More complicated oscillators exist. Here's a period-three oscillator known as "The Pulsar", 


which displays some appealing symmetry.

In [ ]:
X = np.zeros((17, 17))
X[2, 4:7] = 1
X[4:7, 7] = 1
X += X.T
X += X[:, ::-1]
X += X[::-1, :]
animate_fig(X.astype(int),frames=6)

# The "Glider"

* There are other classes of object which oscillate, but also move while oscillating. One of the earliest seen is the "Glider", 
* Which after 4 steps returns to its initial configuration, but shifted by one cell in both the x and y direction. 
* This is a configuration that often emerges from random starting points.

In [ ]:
glider = [[1, 0, 0],
          [0, 1, 1],
          [1, 1, 0]]
X = np.zeros((8, 8))
X[:3, :3] = glider
animate_fig(X.astype(int),frames=32)

# Unbounded Growth


An early question posed about the Game of Life was whether any configurations exist which result in asymptotically unbounded growth. It was quickly found that the answer was yes. Though it wasn't the first discovered, the following is one of the most compact configurations which display unbounded growth. Note that this claim is precisely true only on an infinite game board: using a torroidal (i.e. wrapping) geometry like we do here will lead to different results, but the first several hundred generations are unaffected:

In [ ]:
unbounded = [[1, 1, 1, 0, 1],
             [1, 0, 0, 0, 0],
             [0, 0, 0, 1, 1],
             [0, 1, 1, 0, 1],
             [1, 0, 1, 0, 1]]
X = np.zeros((30, 40))
X[15:20, 18:23] = unbounded
animate_fig(X.astype(int),frames=100)

# The "Gosper Glider Gun" --> my favourite


The earliest known instance of unbounded growth is one of my favorite configurations: the "Glider Gun" discovered by Bill Gosper. It is an oscillating pattern that creates an infinite series of gliders. It still amazes me that something like this can even emerge from Conway's simple rules, but here it is. We'll stop after a couple hundred frames, but given an infinite game board this action would go on forever:

In [ ]:
glider_gun =\
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
 [1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

X = np.zeros((50, 70))
X[1:10,1:37] = glider_gun

animate_fig(X.astype(int), frames=180,interval=50)

### Here are another few patterns you might try embedding in a game board, to see what will happen

In [ ]:
diehard = [[0, 0, 0, 0, 0, 0, 1, 0],
           [1, 1, 0, 0, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 1, 1, 1]]

boat = [[1, 1, 0],
        [1, 0, 1],
        [0, 1, 0]]

r_pentomino = [[0, 1, 1],
               [1, 1, 0],
               [0, 1, 0]]

beacon = [[0, 0, 1, 1],
          [0, 0, 1, 1],
          [1, 1, 0, 0],
          [1, 1, 0, 0]]

acorn = [[0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [1, 1, 0, 0, 1, 1, 1]]

spaceship = [[0, 0, 1, 1, 0],
             [1, 1, 0, 1, 1],
             [1, 1, 1, 1, 0],
             [0, 1, 1, 0, 0]]

block_switch_engine = [[0, 0, 0, 0, 0, 0, 1, 0],
                       [0, 0, 0, 0, 1, 0, 1, 1],
                       [0, 0, 0, 0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 1, 0, 0, 0],
                       [0, 0, 1, 0, 0, 0, 0, 0],
                       [1, 0, 1, 0, 0, 0, 0, 0]]

In [ ]:
animate_fig(block_switch_engine,frames=100,interval=50)

## Work in progress, stay tuned, I will updated as I learn on the way. 
## Please Upvote it , it motivates me to write more Quality content

**<span style="color:Red">If you find this kernel useful, Please Upvote it , it motivates me to write more Quality content**